In [1]:
import numpy as np
import pandas as pd
import warnings
from pandas import cut
from pandas import qcut
from sklearn.preprocessing import OneHotEncoder
warnings.filterwarnings("ignore")

In [2]:
data = pd.read_csv("./vehicles_4000.csv",index_col="Unnamed: 0")
data1 = data.copy()# 浅复制数据

In [3]:
data.info()
# 有多个数据特征缺失严重，这个时候考虑删除数据所在的列，但这个需要设定一个阈值，当数据列中有多少比例是缺失值的时候删除这个列

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4000 entries, 0 to 3999
Data columns (total 25 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            4000 non-null   int64  
 1   url           4000 non-null   object 
 2   region        4000 non-null   object 
 3   region_url    4000 non-null   object 
 4   price         4000 non-null   int64  
 5   year          3999 non-null   float64
 6   manufacturer  3831 non-null   object 
 7   model         3946 non-null   object 
 8   condition     1995 non-null   object 
 9   cylinders     2331 non-null   object 
 10  fuel          3992 non-null   object 
 11  odometer      3385 non-null   float64
 12  title_status  3990 non-null   object 
 13  transmission  3981 non-null   object 
 14  vin           2512 non-null   object 
 15  drive         2847 non-null   object 
 16  size          1052 non-null   object 
 17  type          3101 non-null   object 
 18  paint_color   2665 non-null 

In [4]:
data.sample(5)#随机查看5个数据

,id,url,region,region_url,price,year,manufacturer,model,condition,cylinders,...,drive,size,type,paint_color,image_url,description,county,state,lat,long
3142,7190992817,https://palmsprings.craigslist.org/ctd/d/fonta...,palm springs,https://palmsprings.craigslist.org,19995,2016.0,ford,f250 super duty super cab,NaN,8 cylinders,...,NaN,NaN,pickup,white,https://images.craigslist.org/00H0H_4ZARJA7axP...,This Ford F250 Super Duty Super Cab can be you...,NaN,ca,34.0794,-117.455
850,7186870974,https://bellingham.craigslist.org/ctd/d/bellin...,bellingham,https://bellingham.craigslist.org,16000,2019.0,hyundai,sonata,NaN,NaN,...,NaN,full-size,sedan,black,https://images.craigslist.org/00E0E_a03oIKg1Rm...,2019 Hyundai Sonata SE Offered by: Rairdon...,NaN,wa,48.7567,-122.451
3979,7193655479,https://albuquerque.craigslist.org/ctd/d/edgew...,albuquerque,https://albuquerque.craigslist.org,21995,2017.0,ford,escape titanium,NaN,NaN,...,fwd,NaN,other,white,https://images.craigslist.org/00z0z_kIqFKml1KG...,🔵Easily BUY ONLINE! We will DELIVER the vehi...,NaN,nm,35.0618,-106.192
712,7194904336,https://albuquerque.craigslist.org/cto/d/albuq...,albuquerque,https://albuquerque.craigslist.org,5200,2012.0,ford,fusion s sedan,NaN,NaN,...,NaN,NaN,NaN,NaN,https://images.craigslist.org/00101_hc50FIafZR...,2012 Ford Fusion s Sedan!4d.2.5L.123k.miles!! ...,NaN,nm,35.0726,-106.575
2286,7186405409,https://kpr.craigslist.org/ctd/d/spokane-2007-...,kennewick-pasco-richland,https://kpr.craigslist.org,13980,2007.0,gmc,yukon denali,NaN,8 cylinders,...,4wd,NaN,SUV,white,https://images.craigslist.org/00E0E_5htwPMOoms...,"2007 GMC Yukon Denali -- $13,980 ☎ Call Or ...",NaN,wa,47.6929,-117.412


In [5]:
data.describe()

,id,price,year,odometer,county,lat,long
count,4.000000e+03,4.000000e+03,3999.000000,3.385000e+03,0.0,3988.000000,3988.000000
mean,7.188352e+09,2.001030e+04,2010.144536,8.985422e+04,NaN,40.568278,-106.995927
std,4.098174e+06,1.604683e+05,10.578448,8.748926e+04,NaN,6.959093,15.982171
min,7.179432e+09,0.000000e+00,1900.000000,0.000000e+00,NaN,7.815840,-123.823000
25%,7.185234e+09,5.990000e+03,2007.000000,3.611100e+04,NaN,35.088000,-119.824000
50%,7.187247e+09,1.250000e+04,2013.000000,7.900000e+04,NaN,39.772150,-116.359000
75%,7.191358e+09,2.299900e+04,2017.000000,1.286470e+05,NaN,46.384400,-95.124700
max,7.195198e+09,9.999999e+06,2020.000000,2.564237e+06,NaN,48.970600,-71.977400


In [6]:
data_1 = data.dropna(axis=1,thresh=int(len(data)*.8))#当缺失值有20%的时候删除这个列

In [7]:
data_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4000 entries, 0 to 3999
Data columns (total 17 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            4000 non-null   int64  
 1   url           4000 non-null   object 
 2   region        4000 non-null   object 
 3   region_url    4000 non-null   object 
 4   price         4000 non-null   int64  
 5   year          3999 non-null   float64
 6   manufacturer  3831 non-null   object 
 7   model         3946 non-null   object 
 8   fuel          3992 non-null   object 
 9   odometer      3385 non-null   float64
 10  title_status  3990 non-null   object 
 11  transmission  3981 non-null   object 
 12  image_url     4000 non-null   object 
 13  description   4000 non-null   object 
 14  state         4000 non-null   object 
 15  lat           3988 non-null   float64
 16  long          3988 non-null   float64
dtypes: float64(4), int64(2), object(11)
memory usage: 562.5+ KB


In [8]:
data2 = data_1.dropna()

In [9]:
data2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3225 entries, 0 to 3999
Data columns (total 17 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            3225 non-null   int64  
 1   url           3225 non-null   object 
 2   region        3225 non-null   object 
 3   region_url    3225 non-null   object 
 4   price         3225 non-null   int64  
 5   year          3225 non-null   float64
 6   manufacturer  3225 non-null   object 
 7   model         3225 non-null   object 
 8   fuel          3225 non-null   object 
 9   odometer      3225 non-null   float64
 10  title_status  3225 non-null   object 
 11  transmission  3225 non-null   object 
 12  image_url     3225 non-null   object 
 13  description   3225 non-null   object 
 14  state         3225 non-null   object 
 15  lat           3225 non-null   float64
 16  long          3225 non-null   float64
dtypes: float64(4), int64(2), object(11)
memory usage: 453.5+ KB


In [10]:
data2 = data2.drop(['id','url','region_url','image_url'],axis=1)

In [11]:
data2.sample(5)

,region,price,year,manufacturer,model,fuel,odometer,title_status,transmission,description,state,lat,long
3308,redding,19995,2016.0,chevrolet,colorado,gas,100697.0,clean,automatic,2016 Chevrolet Chevy Colorado LT Ext. Cab 4WD ...,ca,38.2277,-122.6190
176,fargo / moorhead,6200,2008.0,ford,edge sel awd,gas,145955.0,clean,automatic,2009 FORD EDGE SEL SPORT UTILITY 4D AWD AUT...,mn,46.8757,-96.8842
1374,space coast,15994,2019.0,hyundai,elantra gt,gas,38055.0,clean,automatic,Cocoa Hyundai address: 1825 West King St. C...,fl,28.3547,-80.7599
220,fargo / moorhead,23988,2012.0,ford,econoline cargo van,gas,6988.0,clean,automatic,http://www.goodguysmotorco.com2012 Ford Econol...,mn,45.5534,-94.2318
3807,reno / tahoe,11499,2015.0,nissan,altima 2.5 s,gas,92000.0,clean,automatic,"2015 Nissan Altima 2.5 S, automatic, FWD, 4-cy...",ca,39.5255,-119.7570


In [ ]:
# for i in data2.columns:
#     data2[[i]] = data2[[i]].fillna(method='ffill')

In [12]:
data2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3225 entries, 0 to 3999
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   region        3225 non-null   object 
 1   price         3225 non-null   int64  
 2   year          3225 non-null   float64
 3   manufacturer  3225 non-null   object 
 4   model         3225 non-null   object 
 5   fuel          3225 non-null   object 
 6   odometer      3225 non-null   float64
 7   title_status  3225 non-null   object 
 8   transmission  3225 non-null   object 
 9   description   3225 non-null   object 
 10  state         3225 non-null   object 
 11  lat           3225 non-null   float64
 12  long          3225 non-null   float64
dtypes: float64(4), int64(1), object(8)
memory usage: 352.7+ KB


### 对数据进行编码

In [13]:
data2['manufacturer'].value_counts()

ford             619
chevrolet        436
toyota           281
honda            166
jeep             163
nissan           160
gmc              137
bmw              137
dodge            136
ram              123
subaru           104
hyundai           99
mercedes-benz     83
volkswagen        59
chrysler          56
kia               56
mitsubishi        53
cadillac          47
audi              46
lexus             42
mazda             41
buick             36
acura             32
infiniti          22
volvo             18
mini              15
lincoln           14
rover             13
pontiac            8
jaguar             8
saturn             6
mercury            4
ferrari            2
fiat               2
tesla              1
Name: manufacturer, dtype: int64

In [14]:
# 对制造商进行编码
# enc = OneHotEncoder()
# enc.fit(data2[['manufacturer']])
data2 = pd.get_dummies(data=data2,columns=['manufacturer'])

In [15]:
data2.sample(5)

,region,price,year,model,fuel,odometer,title_status,transmission,description,state,...,manufacturer_nissan,manufacturer_pontiac,manufacturer_ram,manufacturer_rover,manufacturer_saturn,manufacturer_subaru,manufacturer_tesla,manufacturer_toyota,manufacturer_volkswagen,manufacturer_volvo
3167,palm springs,6750,2012.0,altima 2.5 sl,gas,130000.0,clean,automatic,2012 Nissan Altima 2.5 SL Sedan only at Desert...,ca,...,1,0,0,0,0,0,0,0,0,0
1519,bellingham,26775,2019.0,escape titanium,gas,10336.0,clean,automatic,Bellingham Ford Lincoln STOCK #: 00763...,wa,...,0,0,0,0,0,0,0,0,0,0
2769,"washington, DC",0,2011.0,sienna,gas,136491.0,clean,automatic,2011 TOYOTA SIENNA LE Offered by: Nation A...,dc,...,0,0,0,0,0,0,0,1,0,0
757,fargo / moorhead,2900,2005.0,civic lx,gas,144000.0,clean,automatic,"Low mileage vehicle (144,400 mil). Engine & tr...",mn,...,0,0,0,0,0,0,0,0,0,0
778,fargo / moorhead,22500,2019.0,taurus,gas,8473.0,clean,automatic,"2019 Ford Taurus SEL -- $22,500 ☎ Call: ...",mn,...,0,0,0,0,0,0,0,0,0,0


In [16]:
data2['fuel'].value_counts()

gas         2895
diesel       190
other         94
hybrid        33
electric      13
Name: fuel, dtype: int64

In [17]:
data2 = pd.get_dummies(data=data2,columns=['fuel'])

In [19]:
data2['title_status'].value_counts()

clean         3119
rebuilt         53
salvage         36
lien            16
parts only       1
Name: title_status, dtype: int64

In [20]:
data2 = pd.get_dummies(data=data2,columns=['title_status'])

In [21]:
data2['transmission'].value_counts()

automatic    2967
manual        203
other          55
Name: transmission, dtype: int64

In [22]:
data2 = pd.get_dummies(data=data2,columns=['transmission'])

In [23]:
data2['state'].value_counts()

wa    1114
ca     668
mn     480
nm     352
fl     178
dc     142
tx      94
va      93
ny      42
nc      31
ga      18
ct      13
Name: state, dtype: int64

In [24]:
data2 = pd.get_dummies(data=data2,columns=['state'])

In [25]:
data2.sample(10)

,region,price,year,model,odometer,description,lat,long,manufacturer_acura,manufacturer_audi,...,state_dc,state_fl,state_ga,state_mn,state_nc,state_nm,state_ny,state_tx,state_va,state_wa
3887,reno / tahoe,20988,2006.0,wrangler,77365.0,2006 Jeep Wrangler Rubicon Offered by: Dia...,37.7693,-122.2390,0,0,...,0,0,0,0,0,0,0,0,0,0
128,northwest CT,6000,2009.0,outback,105880.0,"2009 Subaru Outback 105,880 miles Emissions go...",41.9559,-73.3632,0,0,...,0,0,0,0,0,0,0,0,0,0
2672,albuquerque,10995,2012.0,edge,92723.0,"2012 Ford Edge SEL FWD - $10,995 Year: 2012 M...",35.0865,-106.5640,0,0,...,0,0,0,0,0,1,0,0,0,0
350,winchester,8995,2011.0,edge,141690.0,2011 *** Ford EDGE LIMITED SUV *** Ready To...,39.1453,-78.1890,0,0,...,0,0,0,0,0,0,0,0,1,0
192,fargo / moorhead,25900,2000.0,f-450 7.3 reg cab 11' crysteel contractor dump,102783.0,2000 Ford_ F-450 7.3 Reg Cab 11' Crysteel Cont...,45.5277,-94.0915,0,0,...,0,0,0,1,0,0,0,0,0,0
780,bellingham,29999,2015.0,1500 laramie 4x4 gas,101990.0,2015 Dodge Ram 1500 Laramie 4x4 **Two Owner 4x...,47.8412,-122.2970,0,0,...,0,0,0,0,0,0,0,0,0,1
2152,kennewick-pasco-richland,23998,2018.0,qx30,29607.0,Leskovar Mitsubishi address: 3020 W. Clearw...,46.2138,-119.1610,0,0,...,0,0,0,0,0,0,0,0,0,1
1002,fargo / moorhead,6995,2007.0,suburban,228000.0,Call ☏ (218) 631−2700Kal's Kars 1000 Jefferson...,46.4500,-95.1247,0,0,...,0,0,0,1,0,0,0,0,0,0
3554,reno / tahoe,36595,2013.0,f-150,123808.0,Vaughn Motor SportsAsk for: Rory Vaughn☎ (775)...,39.4975,-119.7900,0,0,...,0,0,0,0,0,0,0,0,0,0
2088,kennewick-pasco-richland,47999,2020.0,silverado 1500 lt 4x4,13052.0,2020 Chevrolet Silverado 1500 LT 4x4 **Clean C...,46.2348,-119.1280,0,0,...,0,0,0,0,0,0,0,0,0,1


###  标签编码

官方文档：
#### https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html#sklearn.preprocessing.LabelEncoder

In [ ]:
from sklearn.preprocessing import LabelEncoder
LBE = LabelEncoder()
LBE.fit(data2.manufacturer)

In [ ]:
LBE.classes_